In [6]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import torch.nn.functional as F
import numpy as np
learning_rate = 1e-4
keep_prob_rate = 0.7 #
max_epoch = 3
BATCH_SIZE = 50

DOWNLOAD_MNIST = False
PATH='./data'
if not(os.path.exists(PATH)) or not os.listdir(PATH):
    # not mnist dir or mnist is empyt dir
    DOWNLOAD_MNIST = True

In [7]:
train_data = torchvision.datasets.MNIST(root=PATH,train=True, transform=torchvision.transforms.ToTensor(), download=DOWNLOAD_MNIST,)
train_loader = Data.DataLoader(dataset = train_data ,batch_size= BATCH_SIZE ,shuffle= True)

test_data = torchvision.datasets.MNIST(root = PATH,train = False)
test_x = Variable(torch.unsqueeze(test_data.test_data,dim  = 1),volatile = True).type(torch.FloatTensor)[:500]/255.
test_y = test_data.test_labels[:500].numpy()

9920512it [00:07, 1362827.76it/s]                                                                                      


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw


32768it [00:00, 112931.56it/s]                                                                                         


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw


1654784it [00:01, 1147665.02it/s]                                                                                      


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw


8192it [00:00, 16720.38it/s]                                                                                           


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw
Processing...
Done!


c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\torchvision\datasets\mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\ipykernel_launcher.py:5: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  """
c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\torchvision\datasets\mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [13]:

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d( # ???
                # patch 7 * 7 ; 1  in channels ; 32 out channels ; ; stride is 1
                # padding style is same(that means the convolution opration's input and output have the same size)
                in_channels=  1    ,  
                out_channels= 32    ,
                kernel_size=  7    ,
                stride=      1     ,
                padding=  (3,3)        ,
            ),
            nn.ReLU(),        # activation function
            nn.MaxPool2d(2),  # pooling operation
        )
        self.conv2 = nn.Sequential( # ???
            # line 1 : convolution function, patch 5*5 , 32 in channels ;64 out channels; padding style is same; stride is 1
            # line 2 : choosing your activation funciont
            # line 3 : pooling operation function.
                nn.Conv2d(
                    in_channels=  32    ,  
                    out_channels= 64    ,
                    kernel_size=  5    ,
                    stride=      1     ,
                    padding=  (2,2)        ,
                ),
                nn.ReLU(),        # activation function
                nn.MaxPool2d(2),  # pooling operation
        )
        self.out1 = nn.Linear( 7*7*64 , 1024 , bias= True)   # full connection layer one

        self.dropout = nn.Dropout(keep_prob_rate)
        self.out2 = nn.Linear(1024,10,bias=True)



    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(-1,7*7*64)  # flatten the output of coonv2 to (batch_size ,32 * 7 * 7)    # ???
        out1 = self.out1(x)
        out1 = F.relu(out1)
        out1 = self.dropout(out1)
        out2 = self.out2(out1)
        output = F.softmax(out2)
        return output


def test(cnn):
    global prediction
    y_pre = cnn(test_x)
    _,pre_index= torch.max(y_pre,1)
    pre_index= pre_index.view(-1)
    prediction = pre_index.data.numpy()
    correct  = np.sum(prediction == test_y)
    return correct / 500.0


def train(cnn):
    optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate )
    loss_func = nn.CrossEntropyLoss()
    for epoch in range(max_epoch):
        for step, (x_, y_) in enumerate(train_loader):
            x ,y= Variable(x_),Variable(y_)
            output = cnn(x)  
            loss = loss_func(output,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if step != 0 and step % 20 ==0:
                print("=" * 10,step,"="*5,"="*5, "test accuracy is ",test(cnn) ,"=" * 10 )


In [14]:
cnn = CNN()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out1): Linear(in_features=3136, out_features=1024, bias=True)
  (dropout): Dropout(p=0.7, inplace=False)
  (out2): Linear(in_features=1024, out_features=10, bias=True)
)


In [15]:
train(cnn)

c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\ipykernel_launcher.py:46: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


========== 20 ===== ===== test accuracy is  0.284 ==========
========== 40 ===== ===== test accuracy is  0.326 ==========
========== 60 ===== ===== test accuracy is  0.456 ==========
========== 80 ===== ===== test accuracy is  0.532 ==========
========== 100 ===== ===== test accuracy is  0.592 ==========
========== 120 ===== ===== test accuracy is  0.644 ==========
========== 140 ===== ===== test accuracy is  0.708 ==========
========== 160 ===== ===== test accuracy is  0.71 ==========
========== 180 ===== ===== test accuracy is  0.738 ==========
========== 200 ===== ===== test accuracy is  0.78 ==========
========== 220 ===== ===== test accuracy is  0.784 ==========
========== 240 ===== ===== test accuracy is  0.806 ==========
========== 260 ===== ===== test accuracy is  0.832 ==========
========== 280 ===== ===== test accuracy is  0.832 ==========
========== 300 ===== ===== test accuracy is  0.872 ==========
========== 320 ===== ===== test accuracy is  0.87 ==========
========== 340 

# 修改为 GPU

In [10]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import torch.nn.functional as F
import numpy as np
learning_rate = 1e-4
keep_prob_rate = 0.7 #
max_epoch = 8
BATCH_SIZE = 500

DOWNLOAD_MNIST = False
PATH='./data'
if not(os.path.exists(PATH)) or not os.listdir(PATH):
    # not mnist dir or mnist is empyt dir
    DOWNLOAD_MNIST = True

In [11]:
train_data = torchvision.datasets.MNIST(root=PATH,train=True, transform=torchvision.transforms.ToTensor(), download=DOWNLOAD_MNIST,)
train_loader = Data.DataLoader(dataset = train_data ,batch_size= BATCH_SIZE ,shuffle= True)

test_data = torchvision.datasets.MNIST(root = PATH,train = False)

In [12]:
device = torch.device("cuda")

In [13]:
test_x = Variable(torch.unsqueeze(test_data.test_data,dim  = 1)).type(torch.FloatTensor)[:500]/255.
test_y = test_data.targets[:500].numpy()
test_x=test_x.to(device)

In [14]:
test_y[0]

7

In [15]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d( # ???
                # patch 7 * 7 ; 1  in channels ; 32 out channels ; ; stride is 1
                # padding style is same(that means the convolution opration's input and output have the same size)
                in_channels=  1    ,  
                out_channels= 32    ,
                kernel_size=  7    ,
                stride=      1     ,
                padding=  (3,3)        ,
            ),
            nn.ReLU(),        # activation function
            nn.MaxPool2d(2),  # pooling operation
        )
        self.conv2 = nn.Sequential( # ???
            # line 1 : convolution function, patch 5*5 , 32 in channels ;64 out channels; padding style is same; stride is 1
            # line 2 : choosing your activation funciont
            # line 3 : pooling operation function.
                nn.Conv2d(
                    in_channels=  32    ,  
                    out_channels= 64    ,
                    kernel_size=  5    ,
                    stride=      1     ,
                    padding=  (2,2)        ,
                ),
                nn.ReLU(),        # activation function
                nn.MaxPool2d(2),  # pooling operation
        )
        self.out1 = nn.Linear( 7*7*64 , 1024 , bias= True)   # full connection layer one

        self.dropout = nn.Dropout(keep_prob_rate)
        self.out2 = nn.Linear(1024,10,bias=True)



    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(-1,7*7*64)  # flatten the output of coonv2 to (batch_size ,32 * 7 * 7)    # ???
        out1 = self.out1(x)
        out1 = F.relu(out1)
        out1 = self.dropout(out1)
        out2 = self.out2(out1)
        output = F.softmax(out2)
        return output


def test(cnn):
    global prediction
    cnn.eval()
    y_pre = cnn(test_x)
    _,pre_index= torch.max(y_pre,1)
    pre_index= pre_index.view(-1)
    prediction = pre_index.data.cpu().numpy()
    correct  = np.sum(prediction == test_y)
    cnn.train()
    return correct / 500.0


def train(cnn):
    optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate )
    loss_func = nn.CrossEntropyLoss()
    for epoch in range(max_epoch):
        for step, (x_, y_) in enumerate(train_loader):
            x ,y= x_.to(device),y_.to(device)
            output = cnn(x)  
            loss = loss_func(output,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if step != 0 and step % 20 ==0:
                print("=" * 10,step,"="*5,"="*5, "test accuracy is ",test(cnn) ,"=" * 10 )


In [16]:
cnn = CNN()
cnn=cnn.to(device)
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out1): Linear(in_features=3136, out_features=1024, bias=True)
  (dropout): Dropout(p=0.7, inplace=False)
  (out2): Linear(in_features=1024, out_features=10, bias=True)
)


In [17]:
train(cnn)

c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\ipykernel_launcher.py:46: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


========== 20 ===== ===== test accuracy is  0.304 ==========
========== 40 ===== ===== test accuracy is  0.61 ==========
========== 60 ===== ===== test accuracy is  0.682 ==========
========== 80 ===== ===== test accuracy is  0.764 ==========
========== 100 ===== ===== test accuracy is  0.826 ==========
========== 20 ===== ===== test accuracy is  0.878 ==========
========== 40 ===== ===== test accuracy is  0.902 ==========
========== 60 ===== ===== test accuracy is  0.914 ==========
========== 80 ===== ===== test accuracy is  0.922 ==========
========== 100 ===== ===== test accuracy is  0.918 ==========
========== 20 ===== ===== test accuracy is  0.936 ==========
========== 40 ===== ===== test accuracy is  0.94 ==========
========== 60 ===== ===== test accuracy is  0.942 ==========
========== 80 ===== ===== test accuracy is  0.942 ==========
========== 100 ===== ===== test accuracy is  0.946 ==========
========== 20 ===== ===== test accuracy is  0.952 ==========
========== 40 ===== ===